In [439]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
# import keras_tuner

In [440]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

# data = data.drop(
#     columns=["release_date", "key", "loudness",
#              "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
#              "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

data = data.drop(
    columns=["release_date", "name"])

data.head(500)

,user_id,track_id,name_x,popularity,duration_ms,explicit,danceability,energy,key,loudness,...,valence,tempo,name_y,city,street,favourite_genres,premium_user,genres,skipped,number_of_matching_genres
0,101,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, folk rock...",False,0
1,103,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,"[album rock, art rock, classic rock, folk rock...",False,0
2,132,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1
3,132,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,0.449,0.749,9,-8.585,...,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1
4,132,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,0.449,0.749,9,-8.585,...,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,119,59Pi9lNy6bAGJz5QJkcGpn,I Woke up This Morning - 2017 Remaster,11,332558,0,0.326,0.735,9,-5.953,...,0.835,174.856,Gaja Kafel,Poznań,pl. Grottgera 57,"[hard rock, alternative metal, singer-songwriter]",False,"[album rock, art rock, blues, blues rock, brit...",False,1
496,101,2e95TYicnEkTgutf6UNOHA,A Slow Song,23,423360,0,0.309,0.557,5,-7.049,...,0.205,174.372,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, classic u...",True,1
497,101,2e95TYicnEkTgutf6UNOHA,A Slow Song,23,423360,0,0.309,0.557,5,-7.049,...,0.205,174.372,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, classic u...",True,1
498,103,2e95TYicnEkTgutf6UNOHA,A Slow Song,23,423360,0,0.309,0.557,5,-7.049,...,0.205,174.372,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,"[album rock, art rock, classic rock, classic u...",False,0


In [441]:
unique_favourite_genres = set()
for genres in data['favourite_genres']:
  for genre in genres:
    unique_favourite_genres.add(genre)

unique_genres = set()
for genres in data['genres']:
  for genre in genres:
    unique_genres.add(genre)

# show all common genres between all favourite_genres and genres
print("unique_favourite_genres", len(unique_favourite_genres))
print("unique_genres", len(unique_genres))
common_genres = unique_favourite_genres.intersection(unique_genres)
print("common genres", common_genres)

# remove genres that are not in favourite_genres
data['genres'] = data['genres'].apply(
    lambda x: [genre for genre in x if genre in unique_favourite_genres])

data.head()

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 46
unique_genres 1766
common genres {'blues rock', 'singer-songwriter', 'pop', 'album rock', 'quiet storm', 'ranchera', 'motown', 'vocal jazz', 'classic rock', 'soul', 'art rock', 'pop rock', 'folk', 'lounge', 'j-pop', 'filmi', 'hard rock', 'new wave', 'mellow gold', 'country rock', 'alternative rock', 'permanent wave', 'new romantic', 'new wave pop', 'latin alternative', 'soft rock', 'rock en espanol', 'c-pop', 'brill building pop', 'adult standards', 'alternative metal', 'mpb', 'turkish pop', 'funk', 'europop', 'hoerspiel', 'latin', 'latin pop', 'latin rock', 'tropical', 'mandopop', 'regional mexican', 'metal', 'psychedelic rock', 'rock', 'dance pop'}


,user_id,track_id,name_x,popularity,duration_ms,explicit,danceability,energy,key,loudness,...,valence,tempo,name_y,city,street,favourite_genres,premium_user,genres,skipped,number_of_matching_genres
0,101,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, psychedel...",False,0
1,103,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,"[album rock, art rock, classic rock, psychedel...",False,0
2,132,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,0.491,0.606,7,-10.172,...,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, psychedel...",False,1
3,132,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,0.449,0.749,9,-8.585,...,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, psychedel...",False,1
4,132,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,0.449,0.749,9,-8.585,...,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, psychedel...",False,1


In [442]:
# Combine genres and favourite_genres
all_genres = list(data['favourite_genres'] + data['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

encoded_all_genres = mlb.fit_transform(all_genres)

# Split encoded_genres into genres and favourite_genres
# encoded_favourite_genres = encoded_all_genres[:, :len(data['favourite_genres'][0])]
# encoded_genres = encoded_all_genres[:, len(data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(data['favourite_genres'])
encoded_genres = mlb.transform(data['genres'])
# encoded_common_genres = mlb.transform(data['common_genres'])

popularity_normalized = data['popularity'].values.reshape(-1, 1)
popularity_normalized = (popularity_normalized - np.min(popularity_normalized)) / (
    np.max(popularity_normalized) - np.min(popularity_normalized))

duration_ms_normalized = data['duration_ms'].values.reshape(-1, 1)
duration_ms_normalized = (duration_ms_normalized - np.min(duration_ms_normalized)) / (
    np.max(duration_ms_normalized) - np.min(duration_ms_normalized))

danceability_normalized = data['danceability'].values.reshape(-1, 1)
danceability_normalized = (danceability_normalized - np.min(danceability_normalized)) / (
    np.max(danceability_normalized) - np.min(danceability_normalized))

energy_normalized = data['energy'].values.reshape(-1, 1)
energy_normalized = (energy_normalized - np.min(energy_normalized)) / (
    np.max(energy_normalized) - np.min(energy_normalized))

number_of_matching_genres_normalized = data['number_of_matching_genres'].values.reshape(-1, 1)
number_of_matching_genres_normalized = (number_of_matching_genres_normalized - np.min(
    number_of_matching_genres_normalized)) / (np.max(number_of_matching_genres_normalized) - np.min(
    number_of_matching_genres_normalized))

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(
    data={'popularity': data['popularity'],
          'popularity_normalized': popularity_normalized.reshape(-1),
          'duration_ms': data['duration_ms'],
          'duration_ms_normalized': duration_ms_normalized.reshape(-1),
          'number_of_matching_genres': data['number_of_matching_genres'],
          'number_of_matching_genres_normalized': number_of_matching_genres_normalized.reshape(-1),
          'skipped': data['skipped']})

df.head(5)

,popularity,popularity_normalized,duration_ms,duration_ms_normalized,number_of_matching_genres,number_of_matching_genres_normalized,skipped
0,34,0.354167,247707,0.092836,0,0.000000,False
1,34,0.354167,247707,0.092836,0,0.000000,False
2,34,0.354167,247707,0.092836,1,0.333333,False
3,35,0.364583,140067,0.046724,1,0.333333,False
4,35,0.364583,140067,0.046724,1,0.333333,False


In [443]:
# TODO uzywac keras tokenizer?

# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_favourite_genres, encoded_genres], axis=1)
# X = np.concatenate(
#     [popularity_normalized, duration_ms_normalized, danceability_normalized, energy_normalized,
#      number_of_matching_genres_normalized], axis=1)

# create df from X
df = pd.DataFrame(data=X)
print(df.shape)
#
# Extract the labels
y = data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_test == 0))
print("Y_train skipped", np.count_nonzero(y_test == 1))
print("Y_train skipped %", np.count_nonzero(y_test == 1) / len(y_test))


(9881, 92)
Y_train not skipped 1257
Y_train skipped 720
Y_train skipped % 0.36418816388467373


In [444]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(92 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(92, activation='relu'))
  model.add(Dropout(0.5))
  # model.add(Dense(1000, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # model.compile(
  #     optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
  #     loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
epochs = 500
batch_size = 64

model = build_model(None)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.tuners.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=50,
#     max_trials=10,
#     executions_per_trial=2,
#     directory='my_dir')


Epoch 1/500
124/124 - 2s - loss: 0.6371 - accuracy: 0.6425 - val_loss: 0.5832 - val_accuracy: 0.6823 - 2s/epoch - 13ms/step
Epoch 2/500
124/124 - 1s - loss: 0.5899 - accuracy: 0.6714 - val_loss: 0.5659 - val_accuracy: 0.6950 - 762ms/epoch - 6ms/step
Epoch 3/500
124/124 - 1s - loss: 0.5738 - accuracy: 0.6917 - val_loss: 0.5612 - val_accuracy: 0.6970 - 712ms/epoch - 6ms/step
Epoch 4/500
124/124 - 1s - loss: 0.5710 - accuracy: 0.6938 - val_loss: 0.5613 - val_accuracy: 0.6925 - 763ms/epoch - 6ms/step
Epoch 5/500
124/124 - 1s - loss: 0.5639 - accuracy: 0.6929 - val_loss: 0.5639 - val_accuracy: 0.6844 - 724ms/epoch - 6ms/step
Epoch 6/500
124/124 - 1s - loss: 0.5573 - accuracy: 0.7019 - val_loss: 0.5624 - val_accuracy: 0.6930 - 711ms/epoch - 6ms/step
Epoch 7/500
124/124 - 1s - loss: 0.5587 - accuracy: 0.7043 - val_loss: 0.5636 - val_accuracy: 0.6899 - 845ms/epoch - 7ms/step
Epoch 8/500
124/124 - 1s - loss: 0.5552 - accuracy: 0.7038 - val_loss: 0.5632 - val_accuracy: 0.6945 - 702ms/epoch - 6ms

In [ ]:
# tuner.search(
#     (X_train, y_train),
#     validation_data=(X_test, y_test),
# )

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [ ]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
